### 04. nn.Module로 구현하는 선형회귀
- 이전까지는 선형 회귀를 좀 더 직접적으로 이해하기 위해 가설, 비용함수를 직접 정의해서 선형 회귀모델을 구현
- 이번에는 파이토치에서 이미 구현되어져 제공되고 있는 함수들을 불러오는 것으로 더 쉽게 선형 회귀 모델을 구현

In [2]:
import torch.nn as nn
model = nn.Linear(input_dim, outpu_dim)

In [3]:
import torch.nn.functional as F
cost = f.mse_loss(prediction, y_train)

#### 1. 단순 선형 회귀 구현하기

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
torch.manual_seed(1)

In [6]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [7]:
# 모델을 선언 및 초기화. 단순 선형 회귀이미로 input_dim=1, output_dim=1
model = nn.Linear(1,1)

위의 torch.nn.Linear의 인자로 1,1을 사용
- 하나의 입력 x에 대해서 하나의 출력 y을 가지므로, 입력 차원과 출력 차원 모두 1을 인수로 사용
- model에는 가중치 W와 편향b가 저장되어 있다.
- 이 값은 model.parameters()라는 함수를 사용하여 불러올 수 있다.

In [8]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


- 2개의 값이 출력되는데 첫번째 값이 W고, 두번째 값이 b에 해당된다.
- 두 값 모두 현재는 랜덤 초기화가 되어져 있다.
- 두 값 모두 학습의 대상이므로 requires_grad = True가 되어져 있다.

In [9]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [10]:
# 전체 훈련 데이터에 대해 경사 하강법을 2000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs+1) : 
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b 업데이트
    optimizer.step()
    
    if epoch % 100==0 : 
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/2000 Cost : 13.103541
Epoch  100/2000 Cost : 0.002791
Epoch  200/2000 Cost : 0.001724
Epoch  300/2000 Cost : 0.001066
Epoch  400/2000 Cost : 0.000658
Epoch  500/2000 Cost : 0.000407
Epoch  600/2000 Cost : 0.000251
Epoch  700/2000 Cost : 0.000155
Epoch  800/2000 Cost : 0.000096
Epoch  900/2000 Cost : 0.000059
Epoch 1000/2000 Cost : 0.000037
Epoch 1100/2000 Cost : 0.000023
Epoch 1200/2000 Cost : 0.000014
Epoch 1300/2000 Cost : 0.000009
Epoch 1400/2000 Cost : 0.000005
Epoch 1500/2000 Cost : 0.000003
Epoch 1600/2000 Cost : 0.000002
Epoch 1700/2000 Cost : 0.000001
Epoch 1800/2000 Cost : 0.000001
Epoch 1900/2000 Cost : 0.000000
Epoch 2000/2000 Cost : 0.000000


In [15]:
# W와 b의 값도 확인
# 임의의 입력 4를 선언
new_var = torch.FloatTensor([[4.0]])

# 입력한 값 4에 대해서 예측값 y를 리턴받아 pred_y에 저장
pred_y = model(new_var) #forward 연산

# y=2x이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 : ", pred_y)

훈련 후 입력이 4일 때의 예측값 :  tensor([[7.9989]], grad_fn=<AddmmBackward>)


In [17]:
# 학습 후의 W와 b 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]


- H(x) 식에 입력 x로부터 예측된 y를 얻는 것을 forward 연산이라고 한다.
    - 학습 전, prediction=model(x_train)은 x_train으로부터 예측값을 리턴하므로 forward 연산이다.
    - 학습 후, pred_y = model(new_var)는 임의의 값 new_var로부터 예측값을 리턴하므로 forward 연산이다.
- 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 것을 backward 연산이라고 한다.
    - cost.backward()는 비용 함수로부터 기울기를 구하라는 의미이며 backward 연산이다.

------------------------------------------------------------------------------------------------------------

#### 2. 다중 선형 회귀 구현하기

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
torch.manual_seed(1)

In [20]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80], 
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [21]:
# 모델 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1
model = nn.Linear(3,1)

In [22]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]


In [25]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [26]:
nb_epochs = 2000
for epoch in range(nb_epochs+1) : 
    
    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)과 동일
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()
    
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost:{:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/2000 Cost:31584.347656
Epoch  100/2000 Cost:4.054821
Epoch  200/2000 Cost:3.807917
Epoch  300/2000 Cost:3.577585
Epoch  400/2000 Cost:3.362663
Epoch  500/2000 Cost:3.162074
Epoch  600/2000 Cost:2.974864
Epoch  700/2000 Cost:2.800040
Epoch  800/2000 Cost:2.636795
Epoch  900/2000 Cost:2.484290
Epoch 1000/2000 Cost:2.341803
Epoch 1100/2000 Cost:2.208674
Epoch 1200/2000 Cost:2.084196
Epoch 1300/2000 Cost:1.967848
Epoch 1400/2000 Cost:1.859023
Epoch 1500/2000 Cost:1.757193
Epoch 1600/2000 Cost:1.661925
Epoch 1700/2000 Cost:1.572760
Epoch 1800/2000 Cost:1.489271
Epoch 1900/2000 Cost:1.411090
Epoch 2000/2000 Cost:1.337826


In [27]:
# 임의의 입력 선언
new_var = torch.FloatTensor([[73, 80, 75]])
# 입력한 값 [73, 80, 74]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.5655]], grad_fn=<AddmmBackward>)


In [28]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.0551, 0.5767, 0.3883]], requires_grad=True), Parameter containing:
tensor([0.2791], requires_grad=True)]
